In [43]:
import pandas as pd
import re
import numpy as np
import random
from pdb import set_trace as bp

# reading csv file
df_orig = pd.read_csv(r"/Users/wasapple/Downloads/UnitedSolutions-DataScientistPosition/Problem 1-dataset.csv")
                     
df = np.array(df_orig)
print(df.shape)


(20505, 4)


In [44]:
cleandf = []
labels = []

for i in range(len(df)):

    if (df[i, 1] != "label"):
        line = df[i, :].tolist()
        word_list = re.split(':|,|#|/| ', line[0])
        for word in word_list:
          if word != '' and word != '-':
            line.append(word.lower())
        
        if line[2] is None:
            line[2] = '-1'
        
        if line[3] is None:
            line[3] = '-1'
            
        cleandf.append(line)
        #print(line)
        
        if df[i, 1] not in labels:
            labels.append(df[i, 1])

    
labels.sort()

In [45]:
class_items = []
for j, label in enumerate(labels):
    templist = []
    for item in cleandf:
        if (item[1] == label):
            templist.append(item)
    class_items.append(templist)
            

sortdf = []
for rd in class_items:
    rd.sort()
    for item in rd:
        sortdf.append(item)

#print(sortdf[10:30])



In [46]:
cleandf = []
for i, rd in enumerate(sortdf):
    cleandf.append(rd)
    #print(cleandf[i])
    rd[0] = rd[0].lower()
    wordlist = rd[4:]
    lastid = len(wordlist)
    lastword = wordlist[lastid - 1]
    if(rd[0].find('catalog') == -1):
        #print(rd[0])
        if (lastword.isdigit()) or ((lastword.replace('$','')).isdigit()):
            
            if (rd[0].rfind('*') != -1):
              
                for ind, wd in enumerate(wordlist):
                    if (wd == '*'):
                        cleandf[i][2] = wordlist[ind-1].lower()
                        cleandf[i][2] = (cleandf[i][2].replace('quantity','')).strip()
                
                        cleandf[i][3] = (wordlist[ind + 1].replace('$','')).strip()
                        
        
            else:
                if lastword.isdigit():
                    cleandf[i][2] = wordlist[lastid - 1];
                   
                
                if (lastword.find('$') != -1):
                    cleandf[i][3] = (lastword.replace('$','')).strip()  
print(len(cleandf))


      
 

20500


In [47]:
x = list(range(len(cleandf)))
random.shuffle(x)
trainlist = []
testlist = []
br = int(len(x) * 0.7)
for i in range(br):
    trainlist.append(cleandf[x[i]])

for i in range(len(cleandf)-br):
    testlist.append(cleandf[x[br+i]])

print(len(trainlist))
print(len(testlist))

# build dic from train
dict = []
wfreq = []
for i in range(len(cleandf)):
    word_list = cleandf[i][4:]
    #print(word_list)
    for wid in word_list:
        if (wid in dict):
            wfreq[dict.index(wid)] += 1
        else:
            dict.append(wid);
            wfreq.append(1)

class_prob = []


for j, label in enumerate(labels):
    templist = []
    for item in cleandf:
        if (item[1] == label):
            templist.append(item)
    class_items.append(templist)
            
#print(class_items[0][0][1:3])

class_word_prob = []
for i, label in enumerate(labels):
    temp_count = np.zeros(len(dict))
    for rd in class_items[i]:
        word_list = rd[4:]
        for wid in word_list:
            if wid in dict:
                temp_count[dict.index(wid)] += 1
                
    class_word_prob.append(temp_count)
    class_prob.append(len(class_items[i]))
   
    class_word_prob[i][:] = np.divide(class_word_prob[i][:], wfreq)
    class_prob[i] /= len(cleandf)
    
print(class_prob)

14349
6151
[0.1975609756097561, 0.17658536585365853, 0.21365853658536585, 0.2097560975609756, 0.20243902439024392]


In [48]:
confusion_table = np.zeros([len(labels), len(labels)]);


predict_accuracy = 0

for rd in testlist:
    
    max = -1;
    testsample_prob = 0
    predict_label = -1
    
    word_list = rd[4:]
    for j, label in enumerate(labels):
        
        for wid in word_list:
            ind = dict.index(wid)
            testsample_prob += class_word_prob[j][ind]     

        testsample_prob = testsample_prob * class_prob[j]
          
        if (testsample_prob > max):
            max = testsample_prob
            predict_label = label
        
            
    if (predict_label == rd[1]):
        predict_accuracy += 1
        confusion_table[labels.index(predict_label),labels.index(predict_label)] += 1
    else:
        confusion_table[labels.index(rd[1]), labels.index(predict_label)] += 1
    
predict_accuracy /= len(testlist)

print(predict_accuracy)
print(confusion_table)

0.9899203381563974
[[1211.    0.    0.    0.    0.]
 [   0. 1006.   49.    4.    9.]
 [   0.    0. 1346.    0.    0.]
 [   0.    0.    0. 1268.    0.]
 [   0.    0.    0.    0. 1258.]]
